# Check the property of device and versions

In [ ]:
# !pip install -U "transformers>=4.43.0"

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
!python --version

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
# Run the following to empty the VRAM:
import gc
import torch
# del model, tokenizer, pipe

# Flush memory
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

# Document Preprocessing

In [ ]:
# use the Class
from embed_from_file import EmbeddingFromFile
# Call the class
pdf_path = ["articles/GettingStarted.pdf", "articles/human-nutrition-text.pdf"]
embedder = EmbeddingFromFile(pdf_path= pdf_path, 
                             embed_path="articles/text_chunks_and_embeddings_df.csv",
                            device = "cuda")

In [ ]:
# Create markeddown text using pymupdf4llm
md_text = embedder.markeddown_text(page_chunks=True, show_progress=True)

In [ ]:
# Creating dictionary with page num, token num, content and sentences using NLP LLM model
pages_text = embedder.page_and_text(md_text,first_page= 1)

In [ ]:
# Creating chunked
chunked = embedder.create_chunk(pages_and_texts = pages_text, slice_size=10)

# Filtered dataset excluding chunks less than certain number of tokens
filtered = embedder.df_with_min_chunk_length(pages_and_chunks=chunked, min_token_length=30)

In [ ]:
# Creating embedding and pandas dataframe and saved the embeddings
df_with_embeddings = embedder.sentence_embeddings(filtered, model="all-mpnet-base-v2")

In [ ]:
# Plotting the stats i.e., chunk index vs token per chunks
embedder.plot_stats()

In [ ]:
import random

random.sample(chunked, k=1)

# RAG - Search and Retrieval

### Load the saved embedded dataframe

In [ ]:
import random

import torch
import numpy as np 
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

# Import texts and embedding df
text_chunks_and_embedding_df = pd.read_csv("articles/text_chunks_and_embeddings_df.csv")

# Convert embedding column back to np.array (it got converted to string when it got saved to CSV)
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

# Convert texts and embedding df to list of dicts
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

# Convert embeddings to torch tensor and send to device (note: NumPy arrays are float64, torch tensors are float32 by default)
embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings.shape

### Load the sentense fransformer model and create embedding from saved CSV file or alternatively, use the previously saved embedding.

In [ ]:
from sentence_transformers import util, SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", 
                                      device=device) # choose the device to load the model to

In [ ]:
# Define helper function to print wrapped text 
import textwrap
from time import perf_counter as timer

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model: SentenceTransformer=embedding_model,
                                n_resources_to_return: int=5,
                                print_time: bool=True):
    """
    Embeds a query with model and returns top k scores and indices from embeddings.
    """
    # Embed the query
    query_embedding = model.encode(query, 
                                   convert_to_tensor=True) 

    # Get dot product scores on embeddings
    start_time = timer()
    dot_scores = util.dot_score(query_embedding, embeddings)[0]
    end_time = timer()

    if print_time:
        print(f"[INFO] Time taken to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

    scores, indices = torch.topk(input=dot_scores, 
                                 k=n_resources_to_return)

    return scores, indices

def print_top_results_and_scores(query: str,
                                 embeddings: torch.tensor,
                                 pages_and_chunks: list[dict]=pages_and_chunks,
                                 n_resources_to_return: int=5):
    """
    Takes a query, retrieves most relevant resources and prints them out in descending order.

    Note: Requires pages_and_chunks to be formatted in a specific way (see above for reference).
    """
    
    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings,
                                                  n_resources_to_return=n_resources_to_return)
    
    print(f"Query: {query}\n")
    print("Results:")
    # Loop through zipped together scores and indicies
    for score, index in zip(scores, indices):
        print(f"Score: {score:.4f}")
        # Print relevant sentence chunk (since the scores are in descending order, the most relevant chunk will be first)
        print_wrapped(pages_and_chunks[index]["sentence_chunk"])
        # Print the page number too so we can reference the textbook further and check the results
        print(f"Page number: {pages_and_chunks[index]['page_number']}")
        print("\n")

In [ ]:
query = "What are the macronutrients, and what roles do they play in the human body?"

# Get just the scores and indices of top related results
scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)
scores, indices

In [ ]:
# Print out the texts of the top scores
print_top_results_and_scores(query=query,
                             embeddings=embeddings)

## Check device 

In [ ]:
# Get GPU available memory
import torch
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes / (2**30))
print(f"Available GPU memory: {gpu_memory_gb} GB")

In [ ]:
# Run the following to empty the VRAM:
import gc
import torch
# del model, tokenizer, pipe

# Flush memory
gc.collect()
torch.cuda.empty_cache()

# Load huggingface Text Generation LLM

In [ ]:
# huggingface-cli download meta-llama/Llama-3.2-1B-Instruct --include "original/*" --local-dir Llama-3.2-1B-Instruct


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Load model
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=False)

### Check the model size and memory requirements

In [ ]:
def get_model_num_params(model: torch.nn.Module):
    return sum([param.numel() for param in model.parameters()])

get_model_num_params(model)

In [ ]:
def get_model_mem_size(model: torch.nn.Module):
    """
    Get how much memory a PyTorch model takes up.

    See: https://discuss.pytorch.org/t/gpu-memory-that-model-uses/56822
    """
    # Get model parameters and buffer sizes
    mem_params = sum([param.nelement() * param.element_size() for param in model.parameters()])
    mem_buffers = sum([buf.nelement() * buf.element_size() for buf in model.buffers()])

    # Calculate various model sizes
    model_mem_bytes = mem_params + mem_buffers # in bytes
    model_mem_mb = model_mem_bytes / (1024**2) # in megabytes
    model_mem_gb = model_mem_bytes / (1024**3) # in gigabytes

    return {"model_mem_bytes": model_mem_bytes,
            "model_mem_mb": round(model_mem_mb, 2),
            "model_mem_gb": round(model_mem_gb, 2)}

get_model_mem_size(model)

# Text Generation using LLM using RAG

In [ ]:
# Nutrition-style questions generated with GPT4
gpt4_questions = [
    "What are the macronutrients, and what roles do they play in the human body?",
    "How do vitamins and minerals differ in their roles and importance for health?",
    "Describe the process of digestion and absorption of nutrients in the human body.",
    "What role does fibre play in digestion? Name five fibre containing foods.",
    "Explain the concept of energy balance and its importance in weight management."
]

# Manually created question list
manual_questions = [
    "How often should infants be breastfed?",
    "What are symptoms of pellagra?",
    "How does saliva help with digestion?",
    "What is the RDI for protein per day?",
    "water soluble vitamins"
]

query_list = gpt4_questions + manual_questions

In [ ]:
import random
# query = random.choice(query_list)
query = query_list[0]

print(f"Query: {query}")

# Get just the scores and indices of top related results
scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)
scores, indices

In [ ]:
def prompt_formatter(query: str, 
                     context_items: list[dict]) -> str:
    """
    Augments query with text-based context from context_items.
    """
    # Join context items into one dotted paragraph
    context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items])

    # Create a base prompt with examples to help the model
    # Note: this is very customizable, I've chosen to use 3 examples of the answer style we'd like.
    # We could also write this in a txt file and import it in if we wanted.
    base_prompt = """
    Based on the following context items, please answer the query.
    Give yourself room to think by extracting relevant passages from the context before answering the query.
    Don't return the thinking, only return the answer.
    Use the following examples as reference for the ideal answer style.
    \nExample 1:
    Query: What are the fat-soluble vitamins?
    Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.
    \nNow use the following context items to answer the user query:
    {context}
    \nRelevant passages: <extract relevant passages from the context here>
    User query: Make sure your answers are as concise as possible. {query}
    Answer:
    """

    # Update base prompt with context items and query   
    base_prompt = base_prompt.format(context=context, query=query)

    # Create prompt template for instruction-tuned model
    dialogue_template = [
        {"role": "user",
        "content": base_prompt}
    ]

    # Apply the chat template
    prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                          tokenize=False,
                                          add_generation_prompt=True)
    return prompt

In [ ]:
def ask(query, 
        temperature=0.7,
        max_new_tokens=512,
        format_answer_text=True, 
        return_answer_only=True):
    """
    Takes a query, finds relevant resources/context and generates an answer to the query based on the relevant resources.
    """
    
    # Get just the scores and indices of top related results
    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings)
    
    # Create a list of context items
    context_items = [pages_and_chunks[i] for i in indices]

    # Add score to context item
    for i, item in enumerate(context_items):
        item["score"] = scores[i].cpu() # return score back to CPU 
        
    # Format the prompt with context items
    prompt = prompt_formatter(query=query,
                              context_items=context_items)
    
    # Tokenize the prompt
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate an output of tokens
    outputs = model.generate(**input_ids,
                                 temperature=temperature,
                                 do_sample=True,
                                 max_new_tokens=max_new_tokens)
    
    # Turn the output tokens into text
    output_text = tokenizer.decode(outputs[0])

    if format_answer_text:
        # Replace special tokens and unnecessary help message
        output_text = output_text.replace(prompt, "").replace("<bos>", "").replace("<eos>", "").replace("Sure, here is the answer to the user query:\n\n", "")

    # Only return the answer without the context items
    if return_answer_only:
        return output_text
    
    return output_text, context_items

In [ ]:
# query = random.choice(query_list)
query = query_list[1]
print(f"Query: {query}")

# Answer query with context and return context 
answer, context_items = ask(query=query, 
                            temperature=0.7,
                            max_new_tokens=512,
                            return_answer_only=False)

print(f"Answer:\n")
print_wrapped(answer)

In [ ]:
print(f"Context items:")
# context_items

# Evaluation

### see this blog about LaaJ: https://arize.com/blog-course/llm-as-a-judge/

How can LLM-as-a-Judge be used for RAG Applications? 

Contextual relevance and faithfulness are two of the most widely-used metrics for assessing the accuracy and relevance of retrieved files of documents when leveraging LLM RAG.

In [ ]:
## Generate a dataframe for all the queries from query_list


eval_df = []
temperature = 0.7  # Define temperature if it's not dynamic

for ii in range(len(query_list)):
    query = query_list[ii]

    # Answer query with context and return context 
    answer, context_items = ask(query=query, 
                                temperature=temperature,
                                max_new_tokens=512,
                                return_answer_only=False)

    eval_df.append({
        "query": query,
        "temperature": temperature,
        "answer": answer.replace('<|im_end|>', '').replace('**', ''),
        "context_items": "\n".join(["".join(item['sentence_chunk']) for item in context_items]) # paragraph
    })

In [ ]:
eval_df = pd.DataFrame(eval_df)
eval_df.head()

In [ ]:
ii = 2
print(eval_df['query'][ii],"\n\n" ,eval_df['answer'][ii])

# Re-check the context from the original Tafsir

In [ ]:
ii = 0
print(eval_df['query'][ii],"\n\n" ,eval_df['answer'][ii], "\n\n" ,eval_df['context_items'][ii])

In [ ]:
import fitz

# Open PDF and load target page
pdf_path = "articles/TafsirIbnKathir.pdf" # requires PDF to be downloaded
doc = fitz.open(pdf_path)
page = doc.load_page(4405 - 1) # number of page 

# Get the image of the page
img = page.get_pixmap(dpi=300)

# Optional: save the image
#img.save("output_filename.png")
doc.close()

# Convert the Pixmap to a numpy array
img_array = np.frombuffer(img.samples_mv, 
                          dtype=np.uint8).reshape((img.h, img.w, img.n))

# Display the image using Matplotlib
import matplotlib.pyplot as plt
plt.figure(figsize=(13, 10))
plt.imshow(img_array)
plt.title(f"Query: '{query}' | Most relevant page:")
plt.axis('off') # Turn off axis
plt.show()

# Use Vector Database

In [2]:
# !pip install faiss-cpu

### Create Indexing store embedding inside vector database

In [ ]:
import faiss

# Convert to numpy (FAISS only works with numpy)
embedding_np = embeddings.cpu().numpy().astype("float32")

# Get dimension of embedding
embedding_dim = embedding_np.shape[1]

# Create the index (Flat = simple, exact search)
index = faiss.IndexFlatL2(embedding_dim)  # Or use IndexFlatIP for dot product
index.add(embedding_np)  # Add all the embeddings to the index

### Search FAISS with your query

In [ ]:
def retrieve_with_faiss(query: str, 
                        index: faiss.Index,
                        model: SentenceTransformer=embedding_model, 
                        k: int=5):
    query_embedding = model.encode(query).astype("float32").reshape(1, -1)
    distances, indices = index.search(query_embedding, k)
    return distances[0], indices[0]

### Print results (same as before!)

In [ ]:
def print_faiss_results(query, index, pages_and_chunks, k=5):
    distances, indices = retrieve_with_faiss(query, index, k=k)
    
    print(f"Query: {query}\n")
    print("Results:")
    for score, idx in zip(distances, indices):
        print(f"Distance: {score:.4f}")
        print_wrapped(pages_and_chunks[idx]['sentence_chunk'])
        print(f"Page number: {pages_and_chunks[idx]['page_number']}")
        print("\n")
